# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [1]:
# !apt-get update && apt-get install -y python3-opencv

In [2]:
# !pip install sklearn

In [3]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 12.0 MB/s 
     |████████████████████████████████| 180 kB 48.3 MB/s 
     |████████████████████████████████| 144 kB 43.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0a0b6d34384a57926628133d4ad46bd94d94d398bc1fa0062599b8294b5fae40
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
# !pip install -q -U albumentations

In [29]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
import wandb
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings(action='ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set Arguments & hyperparameters

In [7]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [8]:
# parameters

### 데이터 디렉토리 설정 ###
# DATA_DIR = '/USER/seongjin'
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글'
NUM_CLS = 2

EPOCHS = 100  # 30
BATCH_SIZE = 60  # 32
LEARNING_RATE = 0.0029  # 0.0005
EARLY_STOPPING_PATIENCE = 30  # 10
INPUT_SHAPE = 299  #128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [9]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        elif self.mode == 'all':
            self.db = self.db
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


In [10]:
# import albumentations as A

# class CustomDataset(Dataset):
#     def __init__(self, data_dir, mode, input_shape):
#         self.data_dir = data_dir
#         self.mode = mode
#         self.input_shape = input_shape

#         # Loading dataset
#         self.db = self.data_loader()

#         # Dataset split
#         if self.mode == 'train':
#             self.db = self.db[:int(len(self.db) * 0.9)]
#         elif self.mode == 'val':
#             self.db = self.db[int(len(self.db) * 0.9):]
#             self.db.reset_index(inplace=True)
#         else:
#             print(f'!!! Invalid split {self.mode}... !!!')

#         ### augmentation added
#         self.aug =  A.Compose([
#             # A.JpegCompression(quality_lower=10, quality_upper=30, always_apply=True, p=0.5),
#             A.HorizontalFlip(p=0.5),
#             A.RandomBrightnessContrast(p=0.5),
#             A.Rotate(limit=40),
#             A.RandomGamma(gamma_limit=(80, 120), p=0.5),
#             A.CLAHE(clip_limit=(1,10), p= 1),
#             # A.ChannelShuffle(),
#             # A.Blur(blur_limit=7, always_apply=False, p=0.5),
#             # A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5)
#         ])
#         # Transform function
#         self.transform = transforms.Compose([transforms.Resize(self.input_shape),
#                                              transforms.ToTensor(),
#                                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
#     def data_loader(self):
#         print('Loading ' + self.mode + ' dataset..')
#         if not os.path.isdir(self.data_dir):
#             print(f'!!! Cannot find {self.data_dir}... !!!')
#             sys.exit()

#         # (COVID : 1, No : 0)
#         db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))

#         return db

#     def __len__(self):
#         return len(self.db)

#     def __getitem__(self, index):
#         data = copy.deepcopy(self.db.loc[index])

#         # Loading image
#         cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
#         if not isinstance(cvimg, np.ndarray):
#             raise IOError("Fail to read %s" % data['file_name'])

#         # Preprocessing images
#         if self.mode == 'train':   ### train일 경우에만 augmentation하도록 지정
#             augment_image = self.aug(image=cvimg)
#             trans_image = self.transform(Image.fromarray(augment_image['image']))
#         elif self.mode == 'val':
#             trans_image = self.transform(Image.fromarray(cvimg))


#         return trans_image, data['COVID']

## Model

In [18]:
class inception_v3(nn.Module):
    def __init__(self, num_cls):
        super(inception_v3, self).__init__()
        self.model = models.inception_v3(aux_logits=False, pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_cls)
        self.dropout = nn.Dropout(0.225)
        self.softmax = nn.Softmax()
    
    def forward(self, inputs):
        output = self.model(inputs)
        output = self.dropout(output)
        output = self.softmax(output)
        return output

## Utils
### EarlyStopper

In [19]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [20]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

        wandb.init(project='COVID Classifier by CT', entity='seongjin_ahn', name='Inception_v3_hyparam')
        wandb.watch(model, loss_fn, log='all', log_freq=10)

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            # print('\npred: ', pred[:])
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        wandb.log({'Epoch': epoch_index, 'loss': self.train_mean_loss, 'acc': self.train_score})
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [21]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [22]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


In [23]:
train_dataset[0][0].shape

torch.Size([3, 299, 299])

#### Load model and other utils

In [25]:
# Load Model
# model = custom_CNN(NUM_CLS).to(DEVICE)
# model = DenseNet(NUM_CLS).to(DEVICE)
model = inception_v3(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
# loss_fn = nn.BCEWithLogitsLoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
model

inception_v3(
  (model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): Basi

### epoch 단위 학습 진행

In [27]:
max_acc = 0

for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    acc = trainer.validation_score
    if acc > max_acc:
        max_acc = acc
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
            }
        torch.save(check_point, '/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글/pts/Inception_v3_hyparam.pt')

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        # check_point = {
        #     'model': model.state_dict(),
        #     'optimizer': optimizer.state_dict(),
        #     'scheduler': scheduler.state_dict()
        # }
        # torch.save(check_point, '/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글/pts/Inception_v3_BCEWithLogitsLoss_lr0005.pt')
        if acc > max_acc:
            max_acc = acc
            check_point = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()
                }
            torch.save(check_point, '/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글/pts/Inception_v3_hyparam.pt')


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Epoch 0, Train loss: 0.9052349130312601, Acc: 0.4612736660929432, F1-Macro: 0.41083259627876545


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Epoch 0, Val loss: 108.33333587646484, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551


  1%|          | 1/100 [04:35<7:34:36, 275.52s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


...

Epoch 64, Val loss: 2.6776551008224487, Acc: 0.7230769230769231, F1-Macro: 0.7149122807017543
Early stopping counter 29/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Epoch 65, Train loss: 0.038364800728029676, Acc: 0.9776247848537005, F1-Macro: 0.9774441030051634


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
 65%|██████▌   | 65/100 [19:57<10:44, 18.43s/it]

Epoch 65, Val loss: 2.4557783603668213, Acc: 0.7538461538461538, F1-Macro: 0.749034749034749
Early stopping counter 30/30
Early stopped


In [28]:
wandb.finish()

Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
acc,▁▁▂▃▃▄▅▅▆▇▇▇▇███▇███████████████████████
loss,██▇▆▆▆▅▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,65
acc,0.97762
loss,0.03836


In [ ]:
all_dataset = CustomDataset(data_dir=DATA_DIR, mode='all', input_shape=INPUT_SHAPE)
all_imgs = []
all_lbls = []
for batch_index, (img, label) in enumerate(DataLoader(all_dataset)):
    img = img.to(DEVICE)
    label = label.to(DEVICE).float()
    
    all_imgs.append(img)
    all_lbls.append(label)
# all_imgs = all_imgs.to(DEVICE)
# all_lbls = all_lbls.to(DEVICE)

## Inference
### 모델 로드

In [30]:
TRAINED_MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글/pts/Inception_v3_hyparam.pt'

### Load dataset

In [31]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [32]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [33]:
# model = inception_v3(NUM_CLS).to(DEVICE)

In [34]:
# model = torch.load(TRAINED_MODEL_PATH)
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])
# model.load_state_dict(torch.load(TRAINED_MODEL_PATH))
model.eval()

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:34, 34.88s/it]

tensor([[9.9931e-01, 6.9025e-04],
        [9.6350e-01, 3.6497e-02],
        [9.9982e-01, 1.7610e-04],
        [2.7736e-01, 7.2264e-01],
        [9.9985e-01, 1.4658e-04],
        [1.7331e-03, 9.9827e-01],
        [1.3462e-05, 9.9999e-01],
        [1.1487e-04, 9.9989e-01],
        [9.9916e-01, 8.4021e-04],
        [2.2371e-04, 9.9978e-01],
        [9.9999e-01, 1.3448e-05],
        [7.6971e-01, 2.3029e-01],
        [9.9977e-01, 2.3208e-04],
        [2.0502e-01, 7.9498e-01],
        [8.2104e-01, 1.7896e-01],
        [9.9945e-01, 5.4879e-04],
        [7.2268e-01, 2.7732e-01],
        [1.0000e+00, 2.5826e-06],
        [9.9740e-01, 2.6037e-03],
        [9.9998e-01, 2.1045e-05],
        [1.0000e+00, 4.1515e-06],
        [9.9996e-01, 4.2757e-05],
        [6.5553e-01, 3.4447e-01],
        [1.0000e+00, 4.2259e-07],
        [9.7434e-03, 9.9026e-01],
        [9.9961e-01, 3.8824e-04],
        [9.9693e-01, 3.0692e-03],
        [7.2148e-02, 9.2785e-01],
        [1.5512e-01, 8.4488e-01],
        [7.525

2it [00:59, 29.85s/it]

tensor([[6.0448e-04, 9.9940e-01],
        [9.1904e-01, 8.0961e-02],
        [9.9568e-01, 4.3201e-03],
        [9.9579e-01, 4.2102e-03],
        [5.4598e-05, 9.9995e-01],
        [1.6590e-05, 9.9998e-01],
        [1.5366e-03, 9.9846e-01],
        [5.4930e-05, 9.9995e-01],
        [9.5257e-01, 4.7430e-02],
        [9.9999e-01, 8.6838e-06],
        [5.2441e-01, 4.7559e-01],
        [9.9215e-01, 7.8488e-03],
        [9.9915e-01, 8.4667e-04],
        [9.9990e-01, 1.0254e-04],
        [9.9893e-07, 1.0000e+00],
        [9.4465e-01, 5.5349e-02],
        [9.9999e-01, 7.7398e-06],
        [9.9918e-01, 8.1791e-04],
        [1.9246e-03, 9.9808e-01],
        [7.4896e-05, 9.9993e-01],
        [2.1965e-03, 9.9780e-01],
        [8.8468e-01, 1.1532e-01],
        [9.8992e-01, 1.0079e-02],
        [1.4700e-02, 9.8530e-01],
        [4.1371e-01, 5.8629e-01],
        [7.0090e-01, 2.9910e-01],
        [9.9946e-01, 5.3864e-04],
        [1.5071e-02, 9.8493e-01],
        [9.9784e-01, 2.1589e-03],
        [4.600

### 결과 저장

In [36]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220209_2차 팀 모의 캐글/predictions/prediction_17_inception_v3_hyparam.csv', index=False)

In [37]:
df

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,1
4,4.png,0
...,...,...
95,95.png,0
96,96.png,1
97,97.png,1
98,98.png,1


In [40]:
sum(pred_lst)

47